# Module 1 Homework

### Question 1: [Index] S&P 500 Stocks Added to the Index

**Which year had the highest number of additions?**

Using the list of S&P 500 companies from Wikipedia's [S&P 500 companies page](https://en.wikipedia.org/wiki/List_of_S%26P_500_companies), download the data including the year each company was added to the index.

Hint: you can use [pandas.read_html](https://pandas.pydata.org/docs/reference/api/pandas.read_html.html) to scrape the data into a DataFrame.

Steps:
1. Create a DataFrame with company tickers, names, and the year they were added.
2. Extract the year from the addition date and calculate the number of stocks added each year.
3. Which year had the highest number of additions (1957 doesn't count, as it was the year when the S&P 500 index was founded)? Write down this year as your answer (the most recent one, if you have several records).

*Context*: 
> "Following the announcement, all four new entrants saw their stock prices rise in extended trading on Friday" - recent examples of S&P 500 additions include DASH, WSM, EXE, TKO in 2025 ([Nasdaq article](https://www.nasdaq.com/articles/sp-500-reshuffle-dash-tko-expe-wsm-join-worth-buying)).

*Additional*: How many current S&P 500 stocks have been in the index for more than 20 years? When stocks are added to the S&P 500, they usually experience a price bump as investors and index funds buy shares following the announcement.

In [1]:
import pandas as pd

In [5]:
url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
table = pd.read_html(url)

In [10]:
df = table[0]
df.head()

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott Laboratories,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888
3,ABBV,AbbVie,Health Care,Biotechnology,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989


In [18]:
df['Date added'] = pd.to_datetime(df['Date added'], errors='coerce')

df['added_year'] = df['Date added'].dt.year


In [28]:
count_by_year = df['added_year'].value_counts()

print(count_by_year)

1957    53
2016    23
2017    23
2019    22
2008    17
2022    16
2024    16
2023    15
2021    15
2012    14
1997    14
2015    14
2018    14
2002    13
2007    12
2009    12
2020    12
1998    11
1976    11
2013    10
2006    10
2011    10
2000     9
1999     9
2010     9
2001     8
2014     8
1994     7
2005     7
1995     7
1985     7
2004     6
1982     5
1989     5
2025     5
1984     5
2003     5
1988     4
1986     3
1993     3
1983     3
1981     3
1992     3
1980     3
1972     2
1987     2
1969     2
1996     2
1965     2
1973     2
1975     2
1970     2
1979     2
1974     1
1991     1
1964     1
1978     1
Name: added_year, dtype: int64


### Question 2. [Macro] Indexes YTD (as of 1 May 2025)

**How many indexes (out of 10) have better year-to-date returns than the US (S&P 500) as of May 1, 2025?**

Using Yahoo Finance World Indices data, compare the year-to-date (YTD) performance (1 January-1 May 2025) of major stock market indexes for the following countries:
* United States - S&P 500 (^GSPC)
* China - Shanghai Composite (000001.SS)
* Hong Kong - HANG SENG INDEX (^HSI)	
* Australia - S&P/ASX 200 (^AXJO)
* India - Nifty 50 (^NSEI)
* Canada - S&P/TSX Composite (^GSPTSE)
* Germany - DAX (^GDAXI)
* United Kingdom - FTSE 100 (^FTSE)
* Japan - Nikkei 225 (^N225)
* Mexico - IPC Mexico (^MXX)
* Brazil - Ibovespa (^BVSP)

*Hint*: use start_date='2025-01-01' and end_date='2025-05-01' when downloading daily data in yfinance

Context: 
> [Global Valuations: Who's Cheap, Who's Not?](https://simplywall.st/article/beyond-the-us-global-markets-after-yet-another-tariff-update) article suggests "Other regions may be growing faster than the US and you need to diversify."

Reference: Yahoo Finance World Indices - https://finance.yahoo.com/world-indices/

*Additional*: How many of these indexes have better returns than the S&P 500 over 3, 5, and 10 year periods? Do you see the same trend?
Note: For simplicity, ignore currency conversion effects.)

In [29]:
import yfinance as yf

In [40]:
indexes = {
    "USA (S&P 500)": "^GSPC",
    "China (Shanghai)": "000001.SS",
    "Hong Kong (Hang Seng)": "^HSI",
    "Australia (ASX 200)": "^AXJO",
    "India (Nifty 50)": "^NSEI",
    "Canada (TSX)": "^GSPTSE",
    "Germany (DAX)": "^GDAXI",
    "UK (FTSE 100)": "^FTSE",
    "Japan (Nikkei 225)": "^N225",
    "Mexico (IPC)": "^MXX",
    "Brazil (Ibovespa)": "^BVSP"
}


dax_daily = yf.download(list(indexes.values()), start = '2025-01-01', end = '2025-05-01')['Close']

[*********************100%***********************]  11 of 11 completed


In [42]:
# Calculate returns YTD
returns = {}

for name, symbol in indexes.items():
    df = dax_daily[symbol].dropna()
    if len(df) > 1:
        ytd_return = (df.iloc[-1] - df.iloc[0]) / df.iloc[0]
        returns[name] = ytd_return

returns_df = pd.DataFrame.from_dict(returns, orient='index', columns=['YTD Return'])
returns_df.sort_values(by='YTD Return', ascending=False, inplace=True)

sp500_return = returns_df.loc["USA (S&P 500)", "YTD Return"]
better_than_sp500 = (returns_df['YTD Return'] > sp500_return).sum()

print(f"Indexes with better returns than S&P 500: {better_than_sp500} of {len(returns_df)-1}")
print("\nSorted returns:")
print(returns_df)

Indexes with better returns than S&P 500: 9 of 10

Sorted returns:
                       YTD Return
Mexico (IPC)             0.130494
Hong Kong (Hang Seng)    0.127200
Brazil (Ibovespa)        0.124387
Germany (DAX)            0.123464
UK (FTSE 100)            0.028426
India (Nifty 50)         0.024904
China (Shanghai)         0.005048
Canada (TSX)            -0.002261
Australia (ASX 200)     -0.009145
USA (S&P 500)           -0.051033
Japan (Nikkei 225)      -0.082979


### Question 3. [Index] S&P 500 Market Corrections Analysis


**Calculate the median duration (in days) of significant market corrections in the S&P 500 index.**

For this task, define a correction as an event when a stock index goes down by **more than 5%** from the closest all-time high maximum.

Steps:
1. Download S&P 500 historical data (1950-present) using yfinance
2. Identify all-time high points (where price exceeds all previous prices)
3. For each pair of consecutive all-time highs, find the minimum price in between
4. Calculate drawdown percentages: (high - low) / high × 100
5. Filter for corrections with at least 5% drawdown
6. Calculate the duration in days for each correction period
7. Determine the 25th, 50th (median), and 75th percentiles for correction durations

*Context:* 
> * Investors often wonder about the typical length of market corrections when deciding "when to buy the dip" ([Reddit discussion](https://www.reddit.com/r/investing/comments/1jrqnte/when_are_you_buying_the_dip/?rdt=64135)).

> * [A Wealth of Common Sense - How Often Should You Expect a Stock Market Correction?](https://awealthofcommonsense.com/2022/01/how-often-should-you-expect-a-stock-market-correction/)

*Hint (use this data to compare with your results)*: Here is the list of top 10 largest corrections by drawdown:
* 2007-10-09 to 2009-03-09: 56.8% drawdown over 517 days
* 2000-03-24 to 2002-10-09: 49.1% drawdown over 929 days
* 1973-01-11 to 1974-10-03: 48.2% drawdown over 630 days
* 1968-11-29 to 1970-05-26: 36.1% drawdown over 543 days
* 2020-02-19 to 2020-03-23: 33.9% drawdown over 33 days
* 1987-08-25 to 1987-12-04: 33.5% drawdown over 101 days
* 1961-12-12 to 1962-06-26: 28.0% drawdown over 196 days
* 1980-11-28 to 1982-08-12: 27.1% drawdown over 622 days
* 2022-01-03 to 2022-10-12: 25.4% drawdown over 282 days
* 1966-02-09 to 1966-10-07: 22.2% drawdown over 240 days

In [43]:
import numpy as np

In [ ]:
# Download S&P 500 data

data = yf.download("^GSPC", start="1950-01-01", end="2025-05-01")

if isinstance(data.columns, pd.MultiIndex):
    data.columns = ['_'.join(col).strip() if col[1] else col[0] for col in data.columns]

data = data.reset_index()

data.rename(columns={'Close': 'Close_GSPC', 'Close_^GSPC': 'Close_GSPC'}, inplace=True)


[*********************100%***********************]  1 of 1 completed


In [90]:
data.columns

Index(['Date', 'Close_GSPC', 'High_^GSPC', 'Low_^GSPC', 'Open_^GSPC',
       'Volume_^GSPC'],
      dtype='object')

In [100]:
# All time high points
data['All_Time_High'] = data['Close_GSPC'].cummax()
high_points = data[data['Close_GSPC'] == data['All_Time_High']].copy().reset_index()

# Find the minimum in between for each consecutive pair of all-time-highs
min_between_highs = []
for i in range(len(high_points) - 1):
    start_idx = high_points.loc[i, 'index']
    end_idx = high_points.loc[i+1, 'index']

    if end_idx - start_idx > 1:
        between = data.loc[start_idx+1:end_idx-1]
        min_row = between.loc[between['Close_GSPC'].idxmin()]
        min_between_highs.append({
            'from_date': data.loc[start_idx, 'Date'],
            'to_date': min_row['Date'],
            'high_price': data.loc[start_idx, 'Close_GSPC'],
            'min_price': min_row['Close_GSPC'],
            'min_date': min_row['Date']
        })

min_df = pd.DataFrame(min_between_highs)

# Calculate Drawdown percentages
min_df['drawdown_pct'] = (min_df['high_price'] - min_df['min_price']) / min_df['high_price'] * 100

# Filter for corrections with at least 5% drawdown
corrections = min_df[min_df['drawdown_pct'] >= 5].copy()

# Calculate duration in days for each correction
corrections['durations_days'] = (corrections['to_date'] - corrections['from_date']).dt.days

# Calculate percentiles for duration
percentiles = corrections['durations_days'].quantile([0.25, 0.5, 0.75]).to_dict()

In [101]:
top = corrections.sort_values(by='drawdown_pct', ascending=False).head(10)

print(top[['from_date', 'to_date', 'drawdown_pct', 'durations_days']])

     from_date    to_date  drawdown_pct  durations_days
448 2007-10-09 2009-03-09     56.775388             517
443 2000-03-24 2002-10-09     49.146948             929
206 1973-01-11 1974-10-03     48.203593             630
193 1968-11-29 1970-05-26     36.061641             543
574 2020-02-19 2020-03-23     33.924960              33
292 1987-08-25 1987-12-04     33.509515             101
133 1961-12-12 1962-06-26     27.973568             196
219 1980-11-28 1982-08-12     27.113582             622
620 2022-01-03 2022-10-12     25.425097             282
176 1966-02-09 1966-10-07     22.177335             240


In [103]:
print(f"The median is {percentiles[0.5]}")

The median is 39.0


### Question 4.  [Stocks] Earnings Surprise Analysis for Amazon (AMZN)


**Calculate the median 2-day percentage change in stock prices following positive earnings surprises days.**

Steps:
1. Load earnings data from CSV ([ha1_Amazon.csv](ha1_Amazon.csv)) containing earnings dates, EPS estimates, and actual EPS. Make sure you are using the correct delimiter to read the data, such as in this command ```python pandas.read_csv("ha1_Amazon.csv", delimiter=';') ```
2. Download complete historical price data using yfinance
3. Calculate 2-day percentage changes for all historical dates: for each sequence of 3 consecutive trading days (Day 1, Day 2, Day 3), compute the *return* as Close_Day3 / Close_Day1 - 1. (Assume Day 2 may correspond to the earnings announcement.)
4. Identify positive earnings surprises (where "actual EPS > estimated EPS" OR "Surprise (%)>0")
5. Calculate 2-day percentage changes following positive earnings surprises. Show your answer in % (closest number to the 2nd digit): *return* * 100.0
6. (Optional) Compare the median 2-day percentage change for positive surprises vs. all historical dates. Do you see the difference?

Context: Earnings announcements, especially when they exceed analyst expectations, can significantly impact stock prices in the short term.

Reference: Yahoo Finance earnings calendar - https://finance.yahoo.com/calendar/earnings?symbol=AMZN

*Additional*: Is there a correlation between the magnitude of the earnings surprise and the stock price reaction? Does the market react differently to earnings surprises during bull vs. bear markets?)


In [166]:
df = pd.read_csv("ha1_Amazon.csv", delimiter=';', encoding='utf-8')

df['EPS Estimate'] = pd.to_numeric(df['EPS Estimate'].str.replace(r'[^\d\.\-]', '', regex=True), errors='coerce')
df['Reported EPS'] = pd.to_numeric(df['Reported EPS'].str.replace(r'[^\d\.\-]', '', regex=True), errors='coerce')

df.head()

,Symbol,Company,Earnings Date,EPS Estimate,Reported EPS,Surprise (%)
0,AMZN,Amazon.com Inc,"April 29, 2026 at 6 AM EDT",NaN,NaN,-
1,AMZN,Amazon.com Inc,"February 4, 2026 at 4 PM EST",NaN,NaN,-
2,AMZN,Amazon.com Inc,"October 29, 2025 at 6 AM EDT",NaN,NaN,-
3,AMZN,Amazon.com Inc,"July 30, 2025 at 4 PM EDT",NaN,NaN,-
4,AMZN,"Amazon.com, Inc.","May 1, 2025 at 4 PM EDT",0.36,0.59,+16.74


In [ ]:
amzn = yf.download("AMZN", start='2020-01-01')

if isinstance(amzn.columns, pd.MultiIndex):
    amzn.columns = [
        f"{col[0]}_{col[1]}" if col[1] else col[0]
        for col in amzn.columns
    ]

amzn = amzn.reset_index()
amzn['Date'] = pd.to_datetime(amzn['Date'])


[*********************100%***********************]  1 of 1 completed


In [168]:
print(amzn.columns.tolist())

['Date', 'Close_AMZN', 'High_AMZN', 'Low_AMZN', 'Open_AMZN', 'Volume_AMZN']


In [ ]:
# Compute 2-day percentage returns
amzn['Return_2d'] = (
    amzn['Close_AMZN'].astype(float)
    .shift(-2)
    .div(amzn['Close_AMZN'].astype(float)) - 1
)

amzn = amzn[amzn['Return_2d'].notna()].copy()

In [170]:
# Convert Earnings Date to datetime
df['Earnings Date'] = pd.to_datetime(df['Earnings Date'], errors='coerce')

# Convert EPS columns to float
df['EPS Estimate'] = pd.to_numeric(df['EPS Estimate'], errors='coerce')
df['Reported EPS'] = pd.to_numeric(df['Reported EPS'], errors='coerce')

# Filter for positive surprises
positive_surprises = df[
    (df['Reported EPS'] > df['EPS Estimate']) &
    df['Earnings Date'].notna()
]

c:\Users\cachu\anaconda3\lib\site-packages\dateutil\parser\_parser.py:1207: UnknownTimezoneWarning: tzname EDT identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "
c:\Users\cachu\anaconda3\lib\site-packages\dateutil\parser\_parser.py:1207: UnknownTimezoneWarning: tzname EST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "


In [ ]:
returns = []

for date in positive_surprises['Earnings Date']:
    future_dates = amzn[amzn['Date'] > date].reset_index(drop=True)
    if len(future_dates) >= 3:
        try:
            p1 = float(future_dates.loc[0, 'Close_AMZN'])
            p3 = float(future_dates.loc[2, 'Close_AMZN'])
            if p1 > 0 and p3 > 0:
                r = (p3 / p1 - 1) * 100
                returns.append(round(r, 2))
        except Exception as e:
            print(f"Skipping {date}: {e}")

print(f"Valid returns: {len(returns)}")
print("Returns (%):", returns)
print("Median return: ", round(pd.Series(returns).median(), 2), "%")

Valid returns: 36
Returns (%): [-2.62, 1.58, 0.79, 4.03, -1.55, 4.19, 0.27, -1.73, -1.24, -0.59, 2.39, 1.16, -4.49, 1.2, 0.4, -0.82, 2.04, 0.26, 0.26, 0.26, 0.26, 0.26, 0.26, 0.26, 0.26, 0.26, 0.26, 0.26, 0.26, 0.26, 0.26, 0.26, 0.26, 0.26, 0.26, 0.26]
Median return:  0.26 %
